Task05 GBDT+LR

### 1. GBDT+LR模型

FM、FFM模型只能做二阶的特征交叉，如果继续提高特征交叉的维度，会不可避免地产生组合爆炸和筛选的问题。Facebook提出了GBDT+LR组合模型的解决方案。利用GBDT自动进行特征筛选和组合，进而生成新的离散特征向量，再把该特征向量当做LR模型输入。

#### 1.1.逻辑回归模型

Logistic Regression的原理比较熟悉了，这里就不赘述了。LR应用到推荐系统中的优势与局限性：

**优点：**

1. LR模型形式简单，可解释性好，从特征的权重可以看到不同的特征对最后结果的影响。
2. 训练时便于并行化，在预测时只需要对特征进行线性加权，所以性能比较好，往往适合处理海量id类特征，用id类特征有一个很重要的好处，就是防止信息损失（相对于范化的 CTR 特征），对于头部资源会有更细致的描述。
3. 资源占用小,尤其是内存。在实际的工程应用中只需要存储权重比较大的特征及特征对应的权重。
4. 方便输出结果调整，调整判定分类的阈值。

**局限性：**

1. 表达能力不强， 无法进行特征交叉， 特征筛选等一系列“高级“操作（需要有经验的工程师来做）， 因此可能造成信息的损失。
2. 准确率并不是很高，模型较简单。
3. 处理非线性数据较麻烦。逻辑回归在不引入其他方法的情况下，只能处理线性可分的数据， 如果想处理非线性， 首先对连续特征的处理需要先进行离散化（离散化的目的是为了引入非线性）。
4. 模型迁移起来比较困难，换一个领域又需要重新进行大量的特征工程。

#### 1.2. GBDT模型

GBDT（Gradient Boosting Decision Tree）梯度提升树，属于Boosting一派的集成学习模型。GBDT通过采用加法模型，即基函数的线性组合，以及不断减小训练过程产生的误差来达到数据分类或者回归。

gbdt通过多轮迭代， 每轮迭代会产生一个弱分类器， 每个分类器在上一轮分类器的残差基础上进行训练gbdt对弱分类器的要求一般是足够简单， 并且低方差高偏差。 因为训练的过程是通过降低偏差来不断提高最终分类器的精度。 由于上述高偏差和简单的要求，每个分类回归树的深度不会很深。最终的总分类器是将每轮训练得到的弱分类器加权求和得到的。

gbdt每轮的训练是在上一轮的训练的残差基础之上进行训练的， 而这里的残差指的就是当前模型的负梯度值，
这个就要求每轮迭代的时候，弱分类器的输出的结果相减是有意义的， 而gbdt无论用于分类还是回归一直都是使用的CART回归树。

**GBDT的优点：** 可以把树的生成过程理解成自动进行多维度的特征组合的过程，从根结点到叶子节点上的整个路径(多个特征值判断)，才能最终决定一棵树的预测值， 另外，对于连续型特征的处理，GBDT 可以拆分出一个临界阈值，比如大于 0.027 走左子树，小于等于 0.027（或者 default 值）走右子树，这样很好的规避了人工离散化的问题。这样就非常轻松的解决了逻辑回归那里自动发现特征并进行有效组合的问题， 这也是GBDT的优势所在。

**局限性：** 对于海量的id类特征，GBDT 由于树的深度和棵树限制（防止过拟合），不能有效的存储；另外海量特征在也会存在性能瓶颈，当 GBDT 的 one hot 特征大于 10 万维时，就必须做分布式的训练才能保证不爆内
存。所以 GBDT 通常配合少量的反馈 CTR 特征来表达，这样虽然具有一定的范化能力，但是同时会有信息损失，对于头部资源不能有效的表达。

#### 1.3 GBDT+LR模型

GBDT+LR组合模型利用GBDT自动进行特征筛选和组合，进而生成新的离散特征向量，再把该特征向量当做LR模型输入。有几个关键点需要注意：

1. 通过GBDT进行特征组合之后得到的离散向量是和训练数据的原特征一块作为逻辑回归的输入， 而不仅仅全是这种离散特征。

2. 建树的时候用ensemble建树的原因就是一棵树的表达能力很弱，不足以表达多个有区分性的特征组合，多棵树的表达能力更强一些。GBDT每棵树都在学习前面棵树尚存的不足，迭代多少次就会生成多少棵树。

3. RF也是多棵树，但从效果上有实践证明不如GBDT。且GBDT前面的树，特征分裂主要体现对多数样本有区分度的特征；后面的树，主要体现的是经过前N颗树，残差仍然较大的少数样本。优先选用在整体上有区分度的特征，再选用针对少数样本有区分度的特征，思路更加合理，这应该也是用GBDT的原因。

4. 在CRT预估中， GBDT一般会建立两类树(非ID特征建一类， ID类特征建一类)， AD，ID类特征在CTR预估中是非常重要的特征，直接将AD，ID作为feature进行建树不可行，故考虑为每个AD，ID建GBDT树。
  1. 非ID类树：不以细粒度的ID建树，此类树作为base，即便曝光少的广告、广告主，仍可以通过此类树得到有区分性的特征、特征组合
  2. ID类树：以细粒度 的ID建一类树，用于发现曝光充分的ID对应有区分性的特征、特征组合

### 2.代码实现

GBDT部分基于Lightgbm实现，所用的数据集是criteo数据集的一部分，有train.csv和test.csv两个文件：

* train.csv： 训练集由Criteo 7天内的部分流量组成。每一行对应一个由Criteo提供的显示广告。为了减少数据集的大小，正(点击)和负(未点击)的例子都以不同的比例进行了抽样。示例是按时间顺序排列的;

* test.csv: 测试集的计算方法与训练集相同，只是针对训练期之后一天的事件。

In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import log_loss

import gc
from scipy import sparse

import warnings
warnings.filterwarnings('ignore')

In [2]:
#定义column名称
cols = ['label',  'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9',
       'I10', 'I11', 'I12', 'I13', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6',
       'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16',
       'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25',
       'C26']

data = pd.read_table('G:\Datasets\dac_sample.tar\dac_sample.txt', header=None, names=cols)

In [3]:
#定义特征组
dense_feats = [f for f in cols if f[0] == "I"]
sparse_feats = [f for f in cols if f[0] == "C"]

In [4]:
#处理dense特征
def process_dense_feats(data, feats):
    d = data.copy()
    d = d[feats].fillna(0.0)   #缺失值填充为0
    for f in feats:
        d[f] = d[f].apply(lambda x : np.log(x + 1) if x > -1 else -1)
        
    return d

In [5]:
data_dense = process_dense_feats(data, dense_feats)

In [6]:
#处理sparse特征
from sklearn.preprocessing import LabelEncoder

def process_sparse_feats(data, feats):
    d = data.copy()
    d = d[feats].fillna("-1")   #缺失值填充为-1
    for f in feats:
        label_encoder = LabelEncoder()    #这里使用了labelEnconder
        d[f] = label_encoder.fit_transform(d[f])
    return d

In [7]:
data_sparse = process_sparse_feats(data, sparse_feats)

In [8]:
#处理后的全量数据
data_df = pd.concat([data_dense, data_sparse], axis=1)
data_df['label'] = data['label']

data_df.shape

(100000, 40)

In [9]:
#划分训练集和测试集
train, test = train_test_split(data_df, test_size=0.2,random_state=0)

In [10]:
y_train = train.pop('label')
x_train = train

y_val = test.pop('label')
x_val = test

#### GBDT建模

In [11]:
gbm = lgb.LGBMClassifier(boosting_type='gbdt',  # 这里用gbdt
                             objective='binary', 
                             subsample=0.8,
                             min_child_weight=0.5, 
                             colsample_bytree=0.7,
                             num_leaves=50,
                             max_depth=6,
                             learning_rate=0.1,
                             n_estimators=1000,
                             silent=True,
                             
                            )

In [12]:
gbm.fit(x_train, y_train, 
        eval_set=[(x_train, y_train), (x_val, y_val)], 
        eval_names=['train', 'val'],
        eval_metric='binary_logloss',
        early_stopping_rounds=100,
        )

[1]	train's binary_logloss: 0.52386	val's binary_logloss: 0.524533
Training until validation scores don't improve for 100 rounds
[2]	train's binary_logloss: 0.514941	val's binary_logloss: 0.515807
[3]	train's binary_logloss: 0.507577	val's binary_logloss: 0.50901
[4]	train's binary_logloss: 0.501155	val's binary_logloss: 0.502763
[5]	train's binary_logloss: 0.496005	val's binary_logloss: 0.497826
[6]	train's binary_logloss: 0.491345	val's binary_logloss: 0.493613
[7]	train's binary_logloss: 0.487444	val's binary_logloss: 0.490113
[8]	train's binary_logloss: 0.484134	val's binary_logloss: 0.487229
[9]	train's binary_logloss: 0.481008	val's binary_logloss: 0.484421
[10]	train's binary_logloss: 0.478329	val's binary_logloss: 0.482231
[11]	train's binary_logloss: 0.475974	val's binary_logloss: 0.480209
[12]	train's binary_logloss: 0.473863	val's binary_logloss: 0.478519
[13]	train's binary_logloss: 0.47174	val's binary_logloss: 0.476791
[14]	train's binary_logloss: 0.469873	val's binary_lo

LGBMClassifier(colsample_bytree=0.7, max_depth=6, min_child_weight=0.5,
               n_estimators=1000, num_leaves=50, objective='binary',
               subsample=0.8)

#### 2.特征转换并构建新的数据集

In [13]:
#获取构建的树
model = gbm.booster_

In [14]:
data_gbdt = pd.concat([x_train, x_val], axis=0)
data_gbdt.shape

(100000, 39)

In [15]:
#每个样本落在每个树的位置，下面的矩阵（样本个数，树的颗数），每一个数字代表某个样本落在了某个树的哪个叶子节点上
gbdt_feats = model.predict(data_gbdt, pred_leaf=True)

In [16]:
gbdt_feats_name = ['gbdt_leaf_' + str(i) for i in range(gbdt_feats.shape[1])]
df_gbdt_feats = pd.DataFrame(gbdt_feats, columns = gbdt_feats_name)

In [17]:
df_gbdt_feats.shape

(100000, 227)

In [18]:
data_gbdt = pd.concat([data_gbdt, df_gbdt_feats], axis = 1)

data_gbdt.shape

(100000, 266)

In [19]:
data_gbdt.head(10)

,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,...,gbdt_leaf_217,gbdt_leaf_218,gbdt_leaf_219,gbdt_leaf_220,gbdt_leaf_221,gbdt_leaf_222,gbdt_leaf_223,gbdt_leaf_224,gbdt_leaf_225,gbdt_leaf_226
0,0.693147,0.693147,1.791759,0.000000,7.232010,1.609438,2.772589,1.098612,5.204007,0.693147,...,24,0,29,0,39,30,12,10,41,44
1,1.098612,0.000000,3.806662,0.693147,4.634729,2.197225,1.098612,1.098612,1.609438,0.693147,...,24,0,29,7,28,31,12,10,9,44
2,1.098612,0.000000,0.693147,2.708050,6.643790,4.499810,1.609438,1.098612,5.505332,0.693147,...,24,0,29,0,31,31,12,0,13,44
3,0.000000,6.795706,0.000000,0.000000,8.387768,0.000000,0.000000,0.000000,0.000000,0.000000,...,23,0,27,7,28,11,12,0,13,12
4,1.386294,-1.000000,0.000000,0.000000,1.098612,0.000000,1.386294,0.000000,0.000000,0.693147,...,24,0,29,31,38,43,20,28,43,43
5,0.000000,-1.000000,0.000000,0.000000,9.459152,0.000000,0.000000,0.000000,1.945910,0.000000,...,24,0,29,0,36,30,1,25,41,42
6,0.000000,0.693147,1.098612,0.000000,8.061171,0.000000,0.000000,0.693147,1.098612,0.000000,...,24,0,29,4,28,31,0,10,13,44
7,0.693147,1.609438,1.098612,0.000000,0.000000,0.000000,0.693147,0.000000,0.000000,0.693147,...,24,0,11,0,36,41,12,28,41,20
8,0.000000,3.806662,1.609438,2.197225,9.852773,5.521461,3.367296,3.465736,4.955827,0.000000,...,24,0,29,33,28,31,12,10,24,44
9,0.000000,3.583519,0.000000,0.693147,10.426380,3.091042,0.693147,1.098612,1.386294,0.000000,...,24,0,29,0,28,37,1,0,0,44


#### 3.将GBDT子树的特征向量做one hot编码

In [20]:
# 新数据的新特征进行one hot编码
for col in gbdt_feats_name:
    onehot_feats = pd.get_dummies(data_gbdt[col], prefix=col)
    data_gbdt.drop([col], axis=1, inplace=True)
    data_gbdt = pd.concat([data_gbdt, onehot_feats], axis=1)

In [22]:
# 连续特征做归一化处理
scaler = MinMaxScaler()
for col in dense_feats:
    data_gbdt[col] = scaler.fit_transform(data_gbdt[col].values.reshape(-1,1))

In [24]:
# 划分数据集
target = data_df['label']
x_train, x_val, y_train, y_val = train_test_split(data_gbdt, target, test_size=0.2,random_state=2020)

In [25]:
x_train.shape

(80000, 9821)

#### 4.训练逻辑回归模型作最后的预测

In [28]:
#训练逻辑回归模型
lr = LogisticRegression()
lr.fit(x_train, y_train)
tr_logloss = log_loss(y_train, lr.predict_proba(x_train)[:,1])
print('tr_logloss:', tr_logloss)
val_logloss = log_loss(y_val, lr.predict_proba(x_val)[:,1])
print('val_logloss:', val_logloss)

tr_logloss: 0.5349669457086674
val_logloss: 0.541720575861628
